In [1]:
from pyspark import SparkContext
sc = SparkContext()

In [2]:
def get_edges(line):
    edge = line.strip().split(',')
    n1 = edge[0]
    n2 = edge[1]
    if n1 < n2:
         return (n1,n2)
    elif n1 > n2:
         return (n2,n1)
    else:
        pass #n1 == n2

def get_rdd_distict_edges(sc, filename):
    return sc.textFile(filename).\
        map(get_edges).\
        filter(lambda x: x is not None).\
        distinct()

def lista_aristas_dep(tupla):
    aristas = []
    for i in range(len(tupla[1])):
        aristas.append(((tupla[0],tupla[1][i]),'exists'))
        for j in range(i+1,len(tupla[1])):
            if tupla[1][i] < tupla[1][j]:
                aristas.append(((tupla[1][i],tupla[1][j]),('pending',tupla[0])))
            else:
                aristas.append(((tupla[1][j],tupla[1][i]),('pending',tupla[0])))
    return aristas

def filtro(tupla):
    return (len(tupla[1]) >=2 and 'exists' in tupla[1])

def triciclos(h):
    l = []
    for i in h[1]:
        if i != 'exists':
            l.append((i[1],h[0][0],h[0][1]))
    return l

def principal(sc,filename):
    edges = get_rdd_distict_edges(sc,filename)
    adj = edges.groupByKey().mapValues(list).flatMap(lista_aristas_dep).\
            groupByKey().mapValues(list).filter(filtro).flatMap(triciclos)
    return adj.collect()

In [4]:
principal(sc,"g4.txt")

[('A', 'B', 'C'), ('A', 'E', 'F'), ('A', 'C', 'E'), ('C', 'D', 'E')]

In [5]:
principal(sc,"g1.txt")

[('A', 'B', 'C'), ('A', 'B', 'D'), ('A', 'C', 'D'), ('B', 'C', 'D')]